#Read Data From SQL

In [ ]:
import pandas as pd
import numpy as np
import psycopg2
import sqlalchemy
import csv
import pyodbc

In [ ]:
server = '192.168.50.52' 
database = 'DefaultEndpoint' 
username = 'ai4bots' 
password = 'AiForBots1!' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [ ]:
data = pd.read_sql_query('''SELECT * FROM DefaultEndpoint.dbo.DeviceHistoryData;''', cnxn)


data.to_csv('DeviceHistoryData.csv')

#Send Dataset to their server

In [ ]:
dataset.to_sql('Name_of_table_in_sql_server', cnxn, schema = 'Name', if_exists='append')

#Calculate the distance

In [ ]:
import math
def calculateDistance(x1,y1,x2,y2):
  
  dy = y2 - y1
  dx = x2 - x1
  dist = math.sqrt((dx)**2 + (dy)**2)
  
  return dist

def get_distance(d):
  v = d.copy()  
  v.drop(columns=['date'], inplace=True)  
  v[['s_long', 's_lat']] = v[['LocationX', 'LocationY']].shift(-1)
  v.dropna(inplace=True)  
  v['distance'] = v[['LocationX', 'LocationY', 's_long', 's_lat']].apply
  (lambda x: calculateDistance(x['s_long'], x['s_lat'], x['LocationX'], x['LocationY']), axis=1)
  return v

#Prepare dataset

In [ ]:
def good_dataset(df):
  
  df_need12 = df12.loc[:,['Time','Alive'	,'LastTimeSeen'	,'ConnectionStage'	,'Enabled'
                     ,'DeviceState'	,'DeviceStateDescription'	,'Errors'	,'Circuit'	,'DestinationNode'	,'LocationX'	,'LocationY']]

  df_need12['Time'] = pd.to_datetime(df_need12['Time'],format='%Y%m%d %H:%M:%S')

  #extract date
  df_need12['date'] = df_need12['Time'].dt.strftime('%D')
  df_need12['date'] = pd.to_datetime(df_need12.date)
  df_need12.insert(0, 'date', df_need12.pop('date'))
  df_need12 = df_need12.sort_values(by='Time')
  df_need12 = df_need12.dropna()  
  df_need12 = df_need12.sort_values(['date', 'Time']).reset_index(drop=True)

  # apply the function to the groups
  test = df_need12.groupby('date').apply(get_distance)
  test = test.loc[:,[	'Time','Enabled'	,'Circuit'	,'DestinationNode'	,'LocationX'	,'LocationY','distance']]

  dpp = test.copy()
  dpp['Time'] = pd.to_datetime(dpp['Time'],format='%Y%m%d %H:%M:%S')
  dpp['Time'] = pd.to_datetime(dpp.Time).dt.tz_localize(None)
  dpp['Time'] = dpp['Time'].astype('datetime64[s]')

  #remove timezone
  dpp['Time'] = pd.to_datetime(dpp.Time).dt.tz_localize(None)

  dpp['T_Time']  = pd.to_datetime(dpp['Time']).dt.time
  dpp['T_Time']  = pd.to_datetime(dpp['Time']).dt.time
  dpp['hour'] = dpp['Time'].dt.hour
  
  #arranging
  zero_column1 = dpp.pop('hour')
  dpp.insert(1, 'hour' ,zero_column1)

  zero_column2 = dpp.pop('T_Time')
  dpp.insert(2, 'T_Time' ,zero_column2)
  return dpp

#Create a Dynamic Matrix

In [ ]:
def get_matrix(df):
  
  dff = df.loc[(df['distance'] == 0)]


  dff = dff.loc[:,['hour','Time','Circuit'	,'LocationX'	,'LocationY','distance']]
  dff = dff.sort_values(by='Time')

  df1 = dff[dff['Circuit'].isin([10, 11, 12, 13, 14, 15])]

  df1 = dff.loc[(dff['Circuit'].ne(dff['Circuit'].shift())) | 
             (dff['Circuit'].ne(dff['Circuit'].shift(-1)))]

  dd = df1.copy()
  ddt = pd.concat([dd.add_suffix('_pre').shift(), dd], axis=1)
  ddt['Time_diff'] = ddt["Time"].diff().apply(lambda x: x/np.timedelta64(1, 's')).fillna(0).astype('int64')

  ds = ddt.loc[:,['hour','Time_pre','Time','Circuit' ,'Time_diff']] 
  ds = ds.dropna()
  dfff = ds.loc[:,['hour','Time_pre','Time','Circuit' ,'Time_diff']]

  df_pivot_avg = pd.pivot_table(dfff, values='Time_diff', index='Circuit', columns=['hour'],aggfunc= np.mean)

  #Transfer row to columns
  df_pivot_avg1 = df_pivot_avg.T
  df_pivot_avg1  = df_pivot_avg1 .reset_index()
  df_pivot_avg1.columns = ['hour','Circuit_10' ,'Circuit_11', 'Circuit_12', 'Circuit_13','Circuit_14','Circuit_15']
  df_circuit_avg = df_pivot_avg1.fillna(0)

  df_circuit_avg['Circuit10_11'] = df_circuit_avg["Circuit_10"] + df_circuit_avg["Circuit_11"]
  df_circuit_avg['Circuit12_13'] = df_circuit_avg["Circuit_12"] + df_circuit_avg["Circuit_13"]
  df_circuit_avg['Circuit14_15'] = df_circuit_avg["Circuit_14"] + df_circuit_avg["Circuit_15"]
  df_circuit_avg = df_circuit_avg.drop(['Circuit_10', 'Circuit_11','Circuit_12', 'Circuit_13', 'Circuit_14', 'Circuit_15'], axis = 1)
  
  return df_circuit_avg